In [ ]:
import pandas as pd
import re
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install Sastrawi
!pip install translate
!pip install tweet-preprocessor
!pip install textblob
!pip install wordcloud
!pip install nltk
!pip install vaderSentiment

# Data Cleaning

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/DATA_ML_kel11/data_mahfud.csv')
df = df[['full_text', 'username', 'created_at']]
df

,full_text,username,created_at
0,Duh! Mahfud MD Ternyata juga Pernah Langgar Et...,omansalamu58888,Mon Dec 25 12:02:11 +0000 2023
1,#Ganjarpranowo ArsjadRasjid mohmahfudmd Cuma p...,alliegrace98,Tue Dec 26 07:29:40 +0000 2023
2,Brarti perusahaan swasta tsb baru menyatakan m...,OZEREV007,Mon Dec 25 11:39:33 +0000 2023
3,Dukungan warga Jatim untuk Ganjar Mahfud dan M...,hanifanur_58,Mon Dec 25 11:39:33 +0000 2023
4,@Fikakhoiriyah97 @officialinews_ Ia dong Ganja...,lloydcabansag,Tue Dec 26 07:29:27 +0000 2023
...,...,...,...
390,"Ganjar Mahfud, semangatmu untuk melayani rakya...",NurSfira,Mon Dec 25 11:27:31 +0000 2023
391,@ex_AnakLolina all in pak Ganjar Mahfud 🤟,rehannnfa,Mon Dec 25 11:27:27 +0000 2023
392,PandamPamungkas Ganjar Pranowo Alam Ganjar sa...,Hariskhattak21t,Mon Dec 25 11:27:26 +0000 2023
393,"Ganjar Mahfud, Gubernur yang berhasil membawa ...",Gilangrmdhan13_,Mon Dec 25 11:27:21 +0000 2023


In [ ]:
df.duplicated().sum()

0

In [ ]:
df = df.dropna()
df.isnull().sum()

full_text     0
username      0
created_at    0
dtype: int64

In [ ]:
def cleanTwitter(text):
  text = re.sub(r'@[A-Za-z0-9_]+', ' ', text)
  text = re.sub(r'#\w+', ' ', text)
  text = re.sub(r'RT[\s]+', ' ', text)
  text = re.sub(r'https?://\S+', ' ', text)
  text = re.sub(r'[^A-Za-z0-9 ]', ' ', text)
  text = re.sub(r'\s+', ' ', text).strip()

  return text

df['full_text'] = df['full_text'].apply(cleanTwitter)
df['full_text'] = df['full_text'].str.lower()

In [ ]:
df = df.drop_duplicates(subset=['full_text'])

In [ ]:
df

,full_text,username,created_at
0,duh mahfud md ternyata juga pernah langgar eti...,omansalamu58888,Mon Dec 25 12:02:11 +0000 2023
1,arsjadrasjid mohmahfudmd cuma pak ganjar mahfu...,alliegrace98,Tue Dec 26 07:29:40 +0000 2023
2,brarti perusahaan swasta tsb baru menyatakan m...,OZEREV007,Mon Dec 25 11:39:33 +0000 2023
3,dukungan warga jatim untuk ganjar mahfud dan m...,hanifanur_58,Mon Dec 25 11:39:33 +0000 2023
4,ia dong ganjar mahfud adalah tim yang berkomit...,lloydcabansag,Tue Dec 26 07:29:27 +0000 2023
...,...,...,...
390,ganjar mahfud semangatmu untuk melayani rakyat...,NurSfira,Mon Dec 25 11:27:31 +0000 2023
391,all in pak ganjar mahfud,rehannnfa,Mon Dec 25 11:27:27 +0000 2023
392,pandampamungkas ganjar pranowo alam ganjar san...,Hariskhattak21t,Mon Dec 25 11:27:26 +0000 2023
393,ganjar mahfud gubernur yang berhasil membawa p...,Gilangrmdhan13_,Mon Dec 25 11:27:21 +0000 2023


# Preprocessing

In [ ]:
# Set the display option
# pd.set_option('display.max_colwidth', None)

for i, text in enumerate(df['full_text']):
  print(f'{i}. {text}')

0. duh mahfud md ternyata juga pernah langgar etika saat jadi ketua mk
1. arsjadrasjid mohmahfudmd cuma pak ganjar mahfud yang begini mendukun umkm agar selalu sukses dan berhasil mahfud hebat
2. brarti perusahaan swasta tsb baru menyatakan minat aja duitnya blm dikasih selama belum ada kontrak resmi brarti bisa saja sewaktu2 perusahaan tsb menarik diri dari investasi pantas saja investor asing blm ada krn investor lokal aja msh lihat kondisi mahfud 100 benar
3. dukungan warga jatim untuk ganjar mahfud dan mahfud md bukan hanya sekadar keinginan untuk melanjutkan tren politik tetapi juga dorongan untuk mendapatkan pemimpin berkualitas
4. ia dong ganjar mahfud adalah tim yang berkomitmen untuk masyarakat 0 0 0 0 in 1728300927977934848 1739269144586166655 ihwan 123 tue dec 26 07 29 26 0000 2023 1739548961596432825 alhamdulillah deklarasi jnk dengan para anjengan amp ustadz muda sepriangan untuk pemenangan ganjar pranowo mahfud md berjalan lancar tanpa hambatan apapun
5. ganjar mahfud kes

In [ ]:
#normalisasi

norm = {
    "gibran": "",
    " kok ": " ",
    " koq ": " ",
    " anies ": " ",
    " cak imin ": " ",
    " muhaimin ": " ",
    " imin ": " ",
    " iy " : " iya ",
    " ya " : " iya ",
    "yohanes" : "",
    "duit" : "uang",
    "mingkem" : "diem",
    " y " : " iya ",
    " ri ": " indonesia ",
    " gak ": " tidak ",
    " gk ": " tidak ",
    " ngk " : " tidak ",
    " yg ": " yang ",
    " pen ": " ingin ",
    " jg ": " juga ",
    "gaterima" : "tidak terima",
    " tlh ": " telah ",
    " ttp ": " tetap ",
    " donk ": " ",
    "menag": "menang",
    " datebg " : " datang ",
    " dateng " : "datang ",
    " ngibul " : " bohong ",
    " tp ": " tetapi ",
    " ud ": " sudah ",
    " lu ": " anda ",
    " ga ": " tidak ",
    " dg ": " dengan ",
    " dgn ": " dengan ",
    " dng ": " dengan ",
    " sdh ": " sudah ",
    " tdk ": " tidak ",
    " tkd ": " tekad ",
    " g ": " tidak ",
    " sodara ": " saudara ",
    " anjay ": " ",
    " anjoy ": " ",
    " bhw ": " bahwa ",
    " jgn " : " jangan ",
    " msh " : " masih ",
    "gw": "saya",
    "ter": " paling ",
    "gua": "saya",
    " tak ": " tidak ",
    " prabowo ": " ",
    " ganjar ": " ",
    " pranowo ": " ",
    "siang": "",
    "iyesssss" : " iya ",
    "selamat natal": "",
    "gamasalah": "tidak masalah",
    " samsul " : " ",
    "denni sauya ": " ",
    " sunarko " : " ",
    "santuy": "santai",
    "lembang": "",
    "deal2an": " deal ",
    "wangsit eyang prabu siliwangi": "",
    "airlangga": "",
    "wa aja": "",
    "palestine": "",
    "tsb": "tersebut",
    "pukis": "",
    "morowali": "",
    " kpd ": " kepada ",
    " kadrun ": " ",
    "emg": " memang ",
    "emg": " emang ",
    "bikin" : "buat",
    "drmn": "darimana",
    " kl ": " jikalau ",
    " jd ": " jadi ",
    " kmrn ": " kemarin ",
    " msk ": " masuk ",
    " sm ": " sama ",
    " sj ": " saja ",
    " aj ": " saja ",
    " aja ": " saja ",
    " ajah ": " saja ",
    " km ": " kamu ",
    " jateng ": " jawa tengah ",
    " gmna ": " gimana ",
    " iatri ": " istri ",
    " temenku ": " teman saya ",
    " turu ": " tidur ",
    " sapiteng ": " ",
    " mas alam ": " ",
    " ntar ": " setelah ini ",
    " ntr ": " setelah ini ",
    "maksa" : "paksa",
    "bakal" : "akan",
    " sdh ": " sudah ",
    "agak" : "sedikit",
    "nampaknya" : "tampak",
    "silahkan" : "silakan",
    "lahhh" : " ",
    " dr " : " dari ",
    " om " : " paman ",
    "grgr": "gara-gara",
    "denger": "dengar",
    "ndak": "tidak",
    "fajrianalfi": "",
    "gemoy" : "lucu",
    "thd" : "terhadap",
    "wbening34496":"",
    " atikoh ":" ",
    "arsjadrasjid mohmahfudmd":"mahfud",
    "kiki daliyo": "",
    "jisatu01":"",
    "tgb zainul majdi ":"",
    "ch chotimah2":"",
    "gus raharjo":"",
    "pandampamungkas":""
}


def normalisasi(text):
  for i in norm:
    text = text.replace(i, norm[i])
  return text

df['full_text'] = df['full_text'].apply(lambda x: normalisasi(x))
df

,full_text,username,created_at
0,duh mahfud md paling nyata juga pernah langga...,omansalamu58888,Mon Dec 25 12:02:11 +0000 2023
1,mahfud cuma pak mahfud yang begini mendukun um...,alliegrace98,Tue Dec 26 07:29:40 +0000 2023
2,brarti perusahaan swasta tersebut baru menyata...,OZEREV007,Mon Dec 25 11:39:33 +0000 2023
3,dukungan warga jatim untuk mahfud dan mahfud m...,hanifanur_58,Mon Dec 25 11:39:33 +0000 2023
4,ia dong mahfud adalah tim yang berkomitmen unt...,lloydcabansag,Tue Dec 26 07:29:27 +0000 2023
...,...,...,...
390,ganjar mahfud semangatmu untuk melayani rakyat...,NurSfira,Mon Dec 25 11:27:31 +0000 2023
391,all in pak mahfud,rehannnfa,Mon Dec 25 11:27:27 +0000 2023
392,alam sangat beruntung mendapatkan didikan yan...,Hariskhattak21t,Mon Dec 25 11:27:26 +0000 2023
393,ganjar mahfud gubernur yang berhasil membawa p...,Gilangrmdhan13_,Mon Dec 25 11:27:21 +0000 2023


In [ ]:
#tokenize
tokenized = df['full_text'].apply(lambda x:x.split())
tokenized

0      [duh, mahfud, md, paling, nyata, juga, pernah,...
1      [mahfud, cuma, pak, mahfud, yang, begini, mend...
2      [brarti, perusahaan, swasta, tersebut, baru, m...
3      [dukungan, warga, jatim, untuk, mahfud, dan, m...
4      [ia, dong, mahfud, adalah, tim, yang, berkomit...
                             ...                        
390    [ganjar, mahfud, semangatmu, untuk, melayani, ...
391                               [all, in, pak, mahfud]
392    [alam, sangat, beruntung, mendapatkan, didikan...
393    [ganjar, mahfud, gubernur, yang, berhasil, mem...
394    [masyaallah, saya, salut, paling, hadap, aa, y...
Name: full_text, Length: 391, dtype: object

In [ ]:
#stemming
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

def stemming(text):
  factory = StemmerFactory()
  stemmer = factory.create_stemmer()
  do = []
  for w in text:
    dt = stemmer.stem(w)
    do.append(dt)
  d_clean = []
  d_clean = " ".join(do)
  return d_clean

tokenized = tokenized.apply(stemming)
tokenized.to_csv("preProcessingMahfud.csv", index=False)

In [ ]:
data = pd.read_csv('preProcessingMahfud.csv')
data

,full_text
0,duh mahfud md paling nyata juga pernah langgar...
1,mahfud cuma pak mahfud yang begini dukun umkm ...
2,brarti usaha swasta sebut baru nyata minat saj...
3,dukung warga jatim untuk mahfud dan mahfud md ...
4,ia dong mahfud adalah tim yang komitmen untuk ...
...,...
386,ganjar mahfud semangat untuk layan rakyat buat...
387,all in pak mahfud
388,alam sangat untung dapat dik yang luar biasa d...
389,ganjar mahfud gubernur yang hasil bawa ubah po...
